# Holoscan Federated Analytics For Medical Devices

This example shows NVIDIA FLARE's integration with NVIDIA Holoscan platform, to enable distributed data analytics for AI applications running on medical devices.

## Introduction

[NVIDIA Holoscan](https://developer.nvidia.com/holoscan-sdk) is an AI sensor processing platform that combines hardware systems for low-latency sensor and network connectivity, optimized libraries for data processing and AI, and core microservices to run streaming, imaging, and other applications, from embedded to edge to cloud. It can be used to build streaming AI pipelines for a variety of domains, including Medical Devices, High Performance Computing at the Edge, Industrial Inspection and more. Holoscan platform offers:
- [Holoscan SDK](https://github.com/nvidia-holoscan/holoscan-sdk): a comprehensive software development kit designed to enable real-time AI processing for streaming data applications. It's an open-source SDK, offering Python and C++ APIs.
- [NVIDIA IGX](https://www.nvidia.com/en-us/edge-computing/products/igx/): an industrial-grade edge AI hardware platform, designed to deliver high performance, advanced functional safety, and robust security for enterprise applications. It is the ideal hardware to run applications developed using Holoscan SDK, delivering optimized performance for time-critical applications, for instance for medical devices.
- [Holoscan Sensor Bridge](https://docs.nvidia.com/holoscan/sensor-bridge/1.0.0/index.html): an FPGA based interface designed to enable real-time, low-latency streaming of data from a wide range of sensors—including cameras, LiDAR, radar, and RF devices, to GPUs.
- [Holohub](https://github.com/nvidia-holoscan/holohub): a central hub for open-source applications developed using Holoscan SDK for different industrial use cases.

![Holoscan](image/holoscan.png)

In this tutorial, we demonstrate how to leverage NVIDIA FLARE to enable federated data analytics for medical devices running on Holoscan. We use the AI [endoscopy out-of-body detection](https://github.com/nvidia-holoscan/holohub/tree/main/applications/endoscopy_out_of_body_detection) Holoscan application as an example, and showcase how to compute federated statistics of the Holoscan application running on a fleet of IGX devices.

![Architecture](image/architecture.png)

- We will start with setting up the Holohub repository, building the endoscopy out-of-body detection Holoscan application and configuring it to output analytic data in a standardized format.
- Then, we will run the application in separate sessions under a simulated environment to generate different analytic data, as if the same application is running on a fleet of IGX devices.
- Next, we will set up NVIDIA FLARE server, admin and clients to perform secure, distributed aggregation of analytic data from the Holoscan application, and compute federated statistics.
- Finally, we will visualize the aggregated statistics from different runs of the Holoscan application.

This tutorial is based on the [official repository for Holoscan Federated Analytics](https://github.com/nvidia-holoscan/holoscan-federated-analytics): refer to this repository to learn more.


## Setting Up the Holoscan Application

Let's first clone Holohub and build the endoscopy out-of-body detection application. Out-of-body detection is an important application in robot-assisted endoscopic surgeries. It helps identify which frames from the endoscope camera are out of the patient body, which might capture personal information about the operating room staff. Storing or broadcasting endoscope camera data without any deidentification might introduce a breach of data privacy regulations. With automatic detection of out-of-body endoscopic frames, a subsequent anonymization of these frames (using, for instance, Gaussian blur or mosaic pixelation) can be applied, achieving automatic deidentification of endoscope camera data. 

The Holoscan endoscopy out-of-body detection application that we will build in this tutorial applies a SEResNet50 deep-learning model, trained on endoscopic data, to perform binary in- and out-of-body classification on every frame of an endoscopic video.

![Model](image/model.png)

To set up the application, let's first clone Holohub with the following command:

In [ ]:
!git clone https://github.com/nvidia-holoscan/holohub.git

After cloning it, let's build an image with basic dependencies to run Holoscan sample applications:

In [ ]:
!cd holohub && ./dev_container build

> **Note**
> - To successfully build Holohub image, you need a PC with Ubuntu operating system and an NVIDIA GPU, or a supported NVIDIA developer kit. Check out the prerequisites [here](https://docs.nvidia.com/holoscan/sdk-user-guide/sdk_installation.html#prerequisites).
> - Depending on your platform, the container build time could take more than 10 minutes.

The command above should build an image named `holohub` with certain tag depending on the current version of Holoscan SDK release, and your platform. For example: `holohub:ngc-v3.3.0-dgpu`. Let's now execute the following command to start a container from the built image:

In [ ]:
!cd holohub && ./dev_container launch

Once inside the container, we can use the `run` script to build the `endoscopy_out_of_body_detection` application:

In [ ]:
# Inside the holohub container
!./run build endoscopy_out_of_body_detection

When building the application for the first time, the `run` script downloads the following from [NVIDIA GPU Cloud](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/clara-holoscan/resources/endoscopy_out_of_body_detection) (NGC) into a `data/endoscopy_out_of_body_detection` folder under `holohub` directory:
- The trained in- and out-of-body detection model in `.onnx` format: `out_of_body_detection.onnx`
- A sample endoscopy video clip in `.mp4` format: `sample_clip_out_of_body_detection.mp4`

The 2 images below show a sample in-body frame (first) and out-of-body frame (second) from the sample video clip.

![In Body](image/in_body_frame.png) ![Out of Body](image/out_body_frame.png)

Once the build is finished, let's try to run this application. The `run` script can be used to run either the C++ or Python version of the application, if they both exist. For instance, execute the following command to run the Python version of the application:

> **Note:** many sample applications from Holohub can be run using either C++ or Python, by specifying `cpp` or `python` flag for the `run` program. In this example, we will run the `endoscopy_out_of_body_detection` in Python. You should get the same result when running it in C++.

In [ ]:
# Inside the holohub container
!./run launch endoscopy_out_of_body_detection python

Upon a successful run, you should see console output as follows, printing the in-/out-of-body classification label and confidence score for each frame of the sample endoscopy video:
```bash
 ...
 Likely in-body. Confidence: 0.780183
 Likely in-body. Confidence: 0.850593
 Likely in-body. Confidence: 0.867589
 Likely in-body. Confidence: 0.715327
 Likely out-of-body. Confidence: 0.628363
 Likely out-of-body. Confidence: 0.814279
 Likely out-of-body. Confidence: 0.873904
 Likely out-of-body. Confidence: 0.922412
...
```

> **Note**: it takes some time to start the application when you run it for the first time. This is because the deep learning model will be converted to optimized TensorRT format based on your platform, which can be time-consuming. 

## Generating Analytics From Holoscan Application

To enable federated analytics for all Holoscan applications, Holoscan SDK provides a `DataExporter` API, allowing applications to export data such as model inference output to a standardized format, for instance, a CSV file.

The `endoscopy_out_of_body_detection` application was configured to leverage `DataExporter` API to output the classification result of each frame to a CSV file marked with the timestamp of the application run. An example output CSV file would have content like the following:
```bash
In-body,Out-of-body,Confidence Score
1,0,0.972435
1,0,0.90207
1,0,0.897973
0,1,0.939281
0,1,0.948691
0,1,0.94994
```

Refer to the [`DataExporter` documentation](https://docs.nvidia.com/holoscan/sdk-user-guide/components/analytics.html#data-exporter-api) to learn more.

Let's now run this application twice, and configure it to generate two distinct output CSV files, simulating a distributed run of the application on 2 distinct IGX devices.

### Running `endoscopy_out_of_body_detection` application on IGX 1

First, let's start the holohub container that we just built, and mount the directory `/tmp/output-igx-1` to generate the application output later. Holoscan applications use the `HOLOSCAN_ANALYTICS_DATA_DIRECTORY` environment variable to determine the analytics output directory. Therefore we need to assign `/tmp/output-igx-1` to `HOLOSCAN_ANALYTICS_DATA_DIRECTORY` variable.

```bash
    cd holohub
    mkdir /tmp/output-igx-1
    ./dev_container launch --docker_opts "-v /tmp/output-igx-1:/workspace/holoscan_data"
    
    # Set the `HOLOSCAN_ANALYTICS_DATA_DIRECTORY` to `/workspace/holoscan_data`
    export HOLOSCAN_ANALYTICS_DATA_DIRECTORY=/workspace/holoscan_data
```

Then, we can run the application with an extra argument `--analytics`. This tells the application to generate a CSV output file inside the previously defined `HOLOSCAN_ANALYTICS_DATA_DIRECTORY` directory.
```bash
    # Run endoscopy out of body detection application.
    ./run launch endoscopy_out_of_body_detection python --extra_args "--analytics"
```

After the run finishes, you should find an output CSV file under `/tmp/output-igx-1/out_of_body_detection/[TIMESTAMP]/data.csv`. `TIMESTAMP` would be the time where the application was executed.

### Running `endoscopy_out_of_body_detection` application on IGX 2

Similar to the previous run, let's simulate running the application on a second IGX, by generating output inside directory `/tmp/output-igx-2`:

```bash
    cd holohub
    mkdir /tmp/output-igx-2
    ./dev_container launch --docker_opts "-v /tmp/output-igx-2:/workspace/holoscan_data"
    
    # Set the `HOLOSCAN_ANALYTICS_DATA_DIRECTORY` to `/workspace/holoscan_data`
    export HOLOSCAN_ANALYTICS_DATA_DIRECTORY=/workspace/holoscan_data
```

Then, we can run the application the same way as before:
```bash
    # Run endoscopy out of body detection application.
    ./run launch endoscopy_out_of_body_detection python --extra_args "--analytics"
```

After the run finishes, you should find an output CSV file under `/tmp/output-igx-2/out_of_body_detection/[TIMESTAMP]/data.csv`.

## Set Up Federated Server and Clients

In this section, we will perform the following to set up the environment to compute federated analytics for the `endoscopy_out_of_body_detection` application using NVIDIA FLARE:
- Build an NVIDIA FLARE image
- Provision a federated system
- Start the server and clients

### Build an NVIDIA FLARE image

To build a docker image with NVIDIA FLARE and dependencies for this tutorial, we use the `dev_container` script provided by the [official repository for Holoscan Federated Analytics](https://github.com/nvidia-holoscan/holoscan-federated-analytics), which we copied under [`code/dev_container`](code/dev_container).

> **Note**: this is not the same `dev_container` script from Holohub repo.

Let's go ahead and execute the following commands in a terminal:

```bash
cd code
./dev_container build
```

This will build an image tagged `holoscan-nvflare-service`, using [code/docker/Dockerfile](code/docker/Dockerfile) as Dockerfile and [code/docker/requirements.txt](code/docker/requirements.txt) for dependencies installation.

### Provision a federated system

Let's first start a container from the image that we just built:
```bash
cd code
docker run -it -v $(pwd):/workspace -u $(id -u):$(id -g) -w /workspace holoscan-nvflare-service 
```

Once inside the container, let's provision a federated system using the configuration file in [code/project.yml](code/project.yml):

```bash
nvflare provision -p project.yml 
```
This will create a `workspace` folder, with a provisioned system for the following participants, as indicated in the configuration file:
- One server with name: `holoscan.nvflare.server`
- Two clients: `Holoscan-Device-1` and `Holoscan-Device-2`. These clients correspond to the two IGX devices that we simulated previously to run the `endoscopy_out_of_body_detection` application separately.
- One project admin with name: `holoscan_admin@nvidia.com`


### Start the server

> **Note**: you can start the server from any machine, as long as NVIDIA FLARE is installed and the machine's IP address is reachable by the clients and the admin. For the sake of simplicity, here we start the server on the same computer as the clients and the admin.

First, let's create a directory to store server-side aggregated output:
```bash
mkdir -p /tmp/output-server/endoscopy_out_of_body_detection
```

Then, we can spin up a container from the `holoscan-nvflare-service` image to start the server. We can do this using `docker run` command with manually specified arguments, but the `dev_container` script provided by the [official repository for Holoscan Federated Analytics](https://github.com/nvidia-holoscan/holoscan-federated-analytics) offers an easier way. Let's go ahead and execute the following command in a terminal:

> **Note**: we are using the `dev_container` script under [`code/dev_container`](code/dev_container). It is not the same `dev_container` script from Holohub repo.

```bash
cd code

# Run NVFLARE server container in a terminal
./dev_container run --server holoscan.nvflare.server --data /tmp/output-server/endoscopy_out_of_body_detection
```
Notice that here we specify `holoscan.nvflare.server` as the server name, as we previously provisioned.

Once inside the container, we can start the NVFLARE server with:
```bash
./start.sh
```
When the server has successfully started, you should see console output similar to:
```bash
...
2025-06-26 09:55:49,392 - root - INFO - Server started
2025-06-26 09:55:54,389 - ServerState - INFO - Got the primary sp: holoscan.nvflare.server fl_port: 8002 SSID: ebc6125d-0a56-4688-9b08-355fe9e4d61a. Turning to hot.
```

### Start the clients

Before starting the client, let's first make sure that the server name `holoscan.nvflare.server` has a reachable IP address inside the `/etc/hosts` of the client machine. For this, we need to add the following to `/etc/hosts` of the clients:
```bash
SERVER_IP_ADDRESS 	holoscan.nvflare.server
```

Since we are simulating everything on a single machine locally, we can just go ahead and add the following to the `/etc/hosts` of the machine
```bash
127.0.0.1 	holoscan.nvflare.server
```

Next, we can leverage the same `dev_container` script to spin up two containers for the two clients, each in a separate terminal.

> **Note**: we are using the `dev_container` script under [`code/dev_container`](code/dev_container). It is not the same `dev_container` script from Holohub repo.

Start a **separate terminal** for client 1:
```bash
cd code
./dev_container run --client Holoscan-Device-1 --data /tmp/output-igx-1
```

Notice that here we specify `Holoscan-Device-1` as the name for client 1, as we previously provisioned. The flag `--data` is used to specify the output directory of the `endoscopy_out_of_body_detection` application, which is `/tmp/output-igx-1` for client 1.

Once inside the container, start the client 1 with:
```bash
./start.sh
```
Upon successful connection to the server, you should see console logs like the following:
```bash
...
2025-06-26 11:59:56,613 - FederatedClient - INFO - Successfully registered client:Holoscan-Device-1 for project holoscan_federated_analytics. Token:f4c2a8e8-65e9-43c1-ae99-1d7f8699ec5c SSID:ebc6125d-0a56-4688-9b08-355fe9e4d61a
2025-06-26 11:59:56,616 - FederatedClient - INFO - Got engine after 0.42764997482299805 seconds
2025-06-26 11:59:56,616 - FederatedClient - INFO - Got the new primary SP: grpc://holoscan.nvflare.server:8002
```

For client 2, the process is similar. Start a **new terminal**, and execute the following commands:
```bash
cd code
./dev_container run --client Holoscan-Device-2 --data /tmp/output-igx-2
```

Once inside the container, start the client 2 with:
```bash
./start.sh
```
Upon successful connection to the server, you should see console logs like the following:
```bash
...
2025-06-26 12:02:24,644 - FederatedClient - INFO - Successfully registered client:Holoscan-Device-2 for project holoscan_federated_analytics. Token:f3bc350d-5b39-4212-ae7d-6c55c8ad2306 SSID:ebc6125d-0a56-4688-9b08-355fe9e4d61a
2025-06-26 12:02:24,649 - FederatedClient - INFO - Got engine after 0.43787550926208496 seconds
2025-06-26 12:02:24,649 - FederatedClient - INFO - Got the new primary SP: grpc://holoscan.nvflare.server:8002
```

## Compute Hierarchical Federated Statistics

The [official repository for Holoscan Federated Analytics](https://github.com/nvidia-holoscan/holoscan-federated-analytics) provides a sample FLARE application `holoscan_fl_example` that computes several federated statistical values from different clients. We will run this sample FLARE application in this tutorial, which is copied under `code/application/holoscan_fl_example`.

First, let's understand what this application is computing. From the [server's configuration](code/application/holoscan_fl_example/app/config/config_fed_server.json), we can see that this application is using the [HierarchicalStatisticsController](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/app_common/workflows/hierarchical_statistics_controller.py), to aggregate the following statistical values from the clients:
- `count`
- `sum`
- `max`
- `min`
- `mean`
- `var` 
- `stddev`
- `histogram`

Local statistics computation is implemented in [code/application/holoscan_fl_example/app/custom/holoscan_statistics.py](code/application/holoscan_fl_example/app/custom/holoscan_statistics.py), which will be used by FLARE's [StatisticsExecutor](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/app_common/executors/statistics/statistics_executor.py), as defined in [code/application/holoscan_fl_example/app/config/config_fed_client.json](code/application/holoscan_fl_example/app/config/config_fed_client.json).

The [HierarchicalStatisticsController](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/app_common/workflows/hierarchical_statistics_controller.py) is **hierarchical** because it exposes a `hierarchy_config` argument to specify the **hierarchy configuration** among all clients. Once a client hierarchy specified, the aggregated statistics will be computed for each hierarchical level. In this sample app, the hierarchy configuration is located at [code/application/holoscan_fl_example/app/config/device_map_2.json](code/application/holoscan_fl_example/app/config/device_map_2.json), with the following content:
```json
{
  "Manufacturers": [
    {
      "Name": "Manufacturer-1",
      "Devices": ["Holoscan-Device-1"]
    },
    {
      "Name": "Manufacturer-2",
      "Devices": ["Holoscan-Device-2"]
    }
  ]
}
```

This configuration defines 3 hierarchical levels: device level, manufacturer level and global level. Therefore aggregated statistics will be computed at:
- Each device individually (this is in fact the local statistics)
- Each manufacturer's devices
- Global level, for all devices from all manufacturers

To learn more about hierarchical statistics, please refer to [this example](https://github.com/NVIDIA/NVFlare/tree/main/examples/advanced/federated-statistics/hierarchical_stats) in NVIDIA FLARE's tutorial catalog, which shows how to compute various statistics under a complex hierarchy.

Now let's run this application. We need to start the admin user and submit a job to run this application.


### Start the admin

> **Note**: you can start the admin from any machine, as long as NVIDIA FLARE is installed and the machine can reach the server. For the sake of simplicity, here we start the admin on the same computer as the server and the clients.

Run the following command in a new terminal to start a container for the admin:

> **Note**: we are using the `dev_container` script under [`code/dev_container`](code/dev_container). It is not the same `dev_container` script from Holohub repo.

```bash
cd code
./dev_container run --admin holoscan_admin@nvidia.com
```

Once inside the container, start the admin with command:
```bash
./startup/fl_admin.sh
```
You will be prompted to enter the `user Name:` of the admin. Enter: `holoscan_admin@nvidia.com`, you will enter the FLARE Console:
```bash
Trying to obtain server address
Obtained server address: holoscan.nvflare.server:8003
Trying to login, please wait ...
Logged into server at holoscan.nvflare.server:8003 with SSID: ebc6125d-0a56-4688-9b08-355fe9e4d61a
Type ? to list commands; type "? cmdName" to show usage of a command.
> 
```


### Submit a job to run the `holoscan_fl_example` application

Let's open a new terminal, and copy the application folder under the `transfer` folder of the admin:
```bash
cp -r code/application/holoscan_fl_example code/workspace/holoscan_federated_analytics/prod_00/holoscan_admin@nvidia.com/transfer/.
```

> **Note:** you can submit the job in FLARE Console by typing the application folder's absolute path. Here for simplicity, we copy the application to admin's transfer folder, so job submission can be done by only typing the application folder's name.

Now, submit a job to run the `holoscan_fl_example` application, by executing the following command inside admin's FLARE Console:

```bash
# Inside FLARE console
submit_job holoscan_fl_example
```

You can use `list_jobs` to query the status of the job's runtime. Once the job is completed, you will get the following status:
```bash
---------------------------------------------------------------------------------------------------------------------------------------------
| JOB ID                               | NAME                      | STATUS             | SUBMIT TIME                      | RUN DURATION   |
---------------------------------------------------------------------------------------------------------------------------------------------
| 443ac73c-261d-4559-adf7-6bbbc868dd38 | holoscan_fl_example_stats | FINISHED:COMPLETED | 2025-06-26T12:16:06.338911+00:00 | 0:00:08.706943 |
---------------------------------------------------------------------------------------------------------------------------------------------
```

The global statistics will be a json file `nvflare_output.json`, available under our predefined output directory `/tmp/output-server/endoscopy_out_of_body_detection/`, under a folder with the timestamp of the current FLARE job execution. 

Examine its content, you will find aggregated global statistics (`count`, `sum`, `max`, `min`, `mean`, `var`, `stddev`, `histogram`), for different hierarchical levels: device (local), manufacturer, and global.

### Visualization of hierarchical statistics

The [official repository for Holoscan Federated Analytics](https://github.com/nvidia-holoscan/holoscan-federated-analytics) also provides [a guide](https://github.com/nvidia-holoscan/holoscan-federated-analytics/blob/main/tutorial/README.md#visualizing-federated-analytics-results) on setting up visualization backend and frontend to visualize federated statistics in a web application.

Let's go ahead and set up the visualization components. The backend and frontend applications are copied under `code/visualization`.

### Set up the backend

First, let's open a terminal, and build and run the backend REST API service container:

> **Note**: the following commands for the backend REST API service need to be run on the same machine as the NVIDIA FLARE server, as the backend container is configured to access our predefined data output directory for the server-side aggregation, i.e., `/tmp/output-server`.

```bash
cd code/visualization/backend
docker compose build
```

This will build an image named `backend-web`. After the build finishes, run the following commands:
```bash
cd code/visualization/backend
docker compose up
```

When the backend service is up-and-running, you should see console logs similar to the following:
```bash
...
analytics-backend  | INFO:     Will watch for changes in these directories: ['/app']
analytics-backend  | INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
analytics-backend  | INFO:     Started reloader process [1] using StatReload
analytics-backend  | INFO:     Started server process [8]
analytics-backend  | INFO:     Waiting for application startup.
analytics-backend  | INFO:     Application startup complete.
```

The REST API service will be accessible on port `8000` by default. Currently the following APIs are supported:
- Getting the list of existing applications (API: `/get_apps/`)
- Getting the list of available statistics for a given application (API: `/get_stats_list/{app_name}/`)
- Getting the statistics for a given application (API: `/get_stats/{app_name}/` and `/get_stats/{app_name}/?timestamp=<timestamp>`)
- Getting the statistics for a date range (API: `/get_range_stats/{app_name}/{start}/{end}/`)

To test the backend API service, let's create a test JWT token that can be used to authenticate REST API service. In a terminal, execute the following commands:
```bash
cd code/visualization/backend

python3 -m venv .venv # You can use whatever python virtual env tool you want to create a virtual env
source .venv/bin/activate
pip3 install -r requirements.txt

TOKEN=`python3 ./create_test_token.py`
echo $TOKEN # We will also use this key later for the frontend.
```
Then, send the following `curl` request in the same terminal:
```bash
curl -H "Authorization: Bearer $TOKEN" http://127.0.0.1:8000/api/v1/get_apps/
```
You should get the console output: `["endoscopy_out_of_body_detection"]`, which is the name of the application that we are running in this tutorial.

### Set up the frontend

Let's build and run the frontend application.

> **Note**: the frontend application can be built and run on any device, as long as the IP address of the backend API service is properly set up (see below). Here for simplicity, we build and run the front end application on the same machine as the backend API service.

First, open the [code/visualization/frontend/analytics-dashboard/.env](code/visualization/frontend/analytics-dashboard/.env) file, and edit the following:
- Set `NEXT_PUBLIC_AUTHORIZATION_HEADER` to be the JWT key that you just created in the section above to test the backend API service. You can print this key with `echo $TOKEN` in the same terminal that you used to generate the JWT key.
- Set `NEXT_PUBLIC_ROOT_URI` to be: `http://[BACKEND_IP_ADDRESS]]:8000/api/v1`, where `BACKEND_IP_ADDRESS`, as the name suggests, is the IP address of the machine running the backend API service. Since we are running everything locally, let's just set it to `http://127.0.0.1:8000/api/v1`.

Open a terminal and run the following commands.

```bash
cd code/visualization/frontend/analytics-dashboard
docker compose build
```

This will build an image named `analytics-dashboard-nextjs`. Now run the frontend application with the following commands:
```bash
cd code/visualization/frontend/analytics-dashboard
docker compose up
```

Once the frontend application is up-and-running, you should see console logs similar to the following:
```bash
...
analytics-dashboard  | > analytics-dashboard@0.1.0 start /app
analytics-dashboard  | > next start -p $PORT
analytics-dashboard  | 
analytics-dashboard  |   ▲ Next.js 14.2.15
analytics-dashboard  |   - Local:        http://localhost:8888
analytics-dashboard  | 
analytics-dashboard  |  ✓ Starting...
analytics-dashboard  |  ✓ Ready in 184ms
...
```

### Visualize the hierarchical statistics

Open a browser and navigate to `http://[FRONTEND_IP_ADDRESS]:8000` (for instance, `http://localhost:8888` if you are using the same machine running the frontend application), you will be able to visualize different aggregated statistics, as illustrated below.

![Visualization](image/visualization.png)

Feel free to play with different settings in the visualization UI to explore more.

## Summary

That's it, you've learned how to leverage NVIDIA FLARE to enable federated analytics for Holoscan applications running on medical devices. Feel free to refer to the [official repository for Holoscan Federated Analytics](https://github.com/nvidia-holoscan/holoscan-federated-analytics) to learn about more aspects to it, for instance, how to [dynamically add new devices to current federation](https://github.com/nvidia-holoscan/holoscan-federated-analytics/blob/main/tutorial/README.md#dynamic-provisioning-of-nvflare-clients-and-users) with FLARE's dynamic provisioning system.
